## How to use Itinerary Builder

Itinerary Builder is a module used to query a database of possible itineraries, and return a dataframe with information on each. The critical outputs include the origins and destinations of each two flight itinerary, their times, airlines, and durations. Included as well is the next best second leg flight for each if the connecting flight is missed.

A machine learning algorithm is used to weight the time cost of a missed connection based on the liklihood of the missed connnection occuring.

Users of the module must provide a connection time assumption. This is the minimum time between connecting flights which the user finds acceptable (e.g., allow 45 minutes minimum for connection).

Examples of each function's usage is below.

In [1]:
from itineraryBuilder import *

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

### Itinerary Builder main function call

With default options, the resulting dataframe is ordered by risk as shown below. Note that "arr_no_later_date" is restricting the results such that itineraries which arrive at their destination the following day are ommitted. This occurs if this parameter equals "flight_date".

In [3]:
origin = 'ATL'
destination = 'SEA'
flight_date = '12/1/2019'

dne = '700'
dnl = '1000'
ane = '1'
anl = '2000'
tc = 45
ane_date = '12/1/2019'
anl_date = '12/2/2019'

df = itineraryBuilder('faa_2019_12', origin, destination, flight_date, ane_date, anl_date, tc, dne, dnl, ane, anl); df

C:\Users\a0084102\OneDrive - dematic.com\Documents\_Personal\GT\CSE6242\project repo\gatech-cse6242-spring2023-project\dataManagement\itineraryBuilder.py:101: PerformanceWarning: Adding/subtracting object-dtype array to TimedeltaArray not vectorized.
  df['FIRST_LEG_ARR_TIMESTAMP'] = df['FIRST_LEG_ARR_TIMESTAMP'] + df['overnight_bool_1'].astype('timedelta64[D]')


,FIRST_LEG_AIRLINE,FIRST_LEG_ORIG,FIRST_LEG_ORIG_CITY,FIRST_LEG_DEST,FIRST_LEG_DEST_CITY,FIRST_LEG_DATE,FIRST_LEG_DEP_TIME,FIRST_LEG_ARR_TIME,FIRST_LEG_PRED15,FIRST_LEG_PRED30,FIRST_LEG_PRED45,FIRST_LEG_PRED60,FIRST_LEG_PRED75,FIRST_LEG_PRED90,FIRST_LEG_PRED105,FIRST_LEG_PRED120,SECOND_LEG_AIRLINE,SECOND_LEG_ORIG,SECOND_LEG_ORIG_CITY,SECOND_LEG_DEST,SECOND_LEG_DEST_CITY,SECOND_LEG_DATE,SECOND_LEG_DEP_TIME,SECOND_LEG_ARR_TIME,NEXT_BEST_SECOND_LEG_DATE,NEXT_BEST_SECOND_LEG_DEP_TIME,NEXT_BEST_SECOND_LEG_ARR_TIME,FIRST_LEG_ORIG_TZ,FIRST_LEG_DEST_TZ,SECOND_LEG_ORIG_TZ,SECOND_LEG_DEST_TZ,FIRST_LEG_DEP_TIMESTAMP,FIRST_LEG_ARR_TIMESTAMP,SECOND_LEG_DEP_TIMESTAMP,SECOND_LEG_ARR_TIMESTAMP,NEXT_BEST_SECOND_LEG_DEP_TIMESTAMP,NEXT_BEST_SECOND_LEG_ARR_TIMESTAMP,overnight_bool_1,overnight_bool_2,overnight_bool_3,FIRST_FLIGHT_DURATION,SECOND_FLIGHT_DURATION,CONNECT_TIME,TRIP_TIME,RISK_MISSED_CONNECTION,NEXT_FLIGHT_TIMELOSS,TOTAL_RISK
0,DL,ATL,"Atlanta, GA",DEN,"Denver, CO",12/1/2019,952,1118,0.009570,0.008820,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,DL,DEN,"Denver, CO",SEA,"Seattle, WA",12/1/2019,1210,1411,12/1/2019,1427,1630,US/Eastern,US/Mountain,US/Mountain,US/Pacific,2019-12-01 09:52:00-05:00,2019-12-01 11:18:00-07:00,2019-12-01 12:10:00-07:00,2019-12-01 14:11:00-08:00,2019-12-01 14:27:00-07:00,2019-12-01 16:30:00-08:00,0,0,0,206.0,181.0,52.0,439.0,0.009570,139.0,1.330291
1,DL,ATL,"Atlanta, GA",PDX,"Portland, OR",12/1/2019,940,1202,0.020706,0.006515,0.010000,0.010000,0.009521,0.000000,0.000000,0.000000,DL,PDX,"Portland, OR",SEA,"Seattle, WA",12/1/2019,1259,1401,12/1/2019,1700,1811,US/Eastern,US/Pacific,US/Pacific,US/Pacific,2019-12-01 09:40:00-05:00,2019-12-01 12:02:00-08:00,2019-12-01 12:59:00-08:00,2019-12-01 14:01:00-08:00,2019-12-01 17:00:00-08:00,2019-12-01 18:11:00-08:00,0,0,0,322.0,62.0,57.0,441.0,0.020706,250.0,5.176559
2,UA,ATL,"Atlanta, GA",DEN,"Denver, CO",12/1/2019,700,829,0.322443,0.460996,0.385243,0.406921,0.404824,0.499367,0.399324,0.434221,UA,DEN,"Denver, CO",SEA,"Seattle, WA",12/1/2019,930,1134,12/1/2019,1210,1411,US/Eastern,US/Mountain,US/Mountain,US/Pacific,2019-12-01 07:00:00-05:00,2019-12-01 08:29:00-07:00,2019-12-01 09:30:00-07:00,2019-12-01 11:34:00-08:00,2019-12-01 12:10:00-07:00,2019-12-01 14:11:00-08:00,0,0,0,209.0,184.0,61.0,454.0,0.460996,157.0,72.376443
3,AA,ATL,"Atlanta, GA",DFW,"Dallas/Fort Worth, TX",12/1/2019,1000,1136,0.569773,0.182787,0.252988,0.000000,0.000000,0.000000,0.000000,0.000000,AA,DFW,"Dallas/Fort Worth, TX",SEA,"Seattle, WA",12/1/2019,1226,1459,12/1/2019,1445,1713,US/Eastern,US/Central,US/Central,US/Pacific,2019-12-01 10:00:00-05:00,2019-12-01 11:36:00-06:00,2019-12-01 12:26:00-06:00,2019-12-01 14:59:00-08:00,2019-12-01 14:45:00-06:00,2019-12-01 17:13:00-08:00,0,0,0,156.0,273.0,50.0,479.0,0.569773,134.0,76.349583
4,UA,ATL,"Atlanta, GA",IAH,"Houston, TX",12/1/2019,730,853,0.010000,0.159435,0.091498,0.087350,0.000000,0.000000,0.000000,0.000000,UA,IAH,"Houston, TX",SEA,"Seattle, WA",12/1/2019,938,1230,12/1/2019,1420,1716,US/Eastern,US/Central,US/Central,US/Pacific,2019-12-01 07:30:00-05:00,2019-12-01 08:53:00-06:00,2019-12-01 09:38:00-06:00,2019-12-01 12:30:00-08:00,2019-12-01 14:20:00-06:00,2019-12-01 17:16:00-08:00,0,0,0,143.0,292.0,45.0,480.0,0.010000,286.0,2.860000
5,AA,ATL,"Atlanta, GA",DFW,"Dallas/Fort Worth, TX",12/1/2019,753,929,0.043448,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,AA,DFW,"Dallas/Fort Worth, TX",SEA,"Seattle, WA",12/1/2019,1032,1305,12/1/2019,1226,1459,US/Eastern,US/Central,US/Central,US/Pacific,2019-12-01 07:53:00-05:00,2019-12-01 09:29:00-06:00,2019-12-01 10:32:00-06:00,2019-12-01 13:05:00-08:00,2019-12-01 12:26:00-06:00,2019-12-01 14:59:00-08:00,0,0,0,156.0,273.0,63.0,492.0,0.000000,114.0,0.000000
6,UA,ATL,"Atlanta, GA",IAD,"Washington, DC",12/1/2019,1000,1143,0.419919,0.169115,0.264511,0.269606,0.322412,0.193112,0.199119,0.179714,UA,IAD,"Washington, DC",SEA,"Seattle, WA",12/1/2019,1230,1523,12/1/2019,1713,2014,US/Eastern,US/Eastern,US/Eas

In [4]:
origin = 'ATL'
destination = 'SEA'
flight_date = '12/1/2019'

dne = '700'
dnl = '1000'
ane = '1'
anl = '2000'
tc = 45
ane_date = '12/1/2019'
anl_date = '12/2/2019'

df = queryFlights('faa_2019_12', origin, destination, flight_date, ane_date, anl_date, dne, dnl, ane, anl); df

,FIRST_LEG_AIRLINE,FIRST_LEG_ORIG,FIRST_LEG_ORIG_CITY,FIRST_LEG_DEST,FIRST_LEG_DEST_CITY,FIRST_LEG_DATE,FIRST_LEG_DEP_TIME,FIRST_LEG_ARR_TIME,FIRST_LEG_PRED15,FIRST_LEG_PRED30,FIRST_LEG_PRED45,FIRST_LEG_PRED60,FIRST_LEG_PRED75,FIRST_LEG_PRED90,FIRST_LEG_PRED105,FIRST_LEG_PRED120,SECOND_LEG_AIRLINE,SECOND_LEG_ORIG,SECOND_LEG_ORIG_CITY,SECOND_LEG_DEST,SECOND_LEG_DEST_CITY,SECOND_LEG_DATE,SECOND_LEG_DEP_TIME,SECOND_LEG_ARR_TIME,NEXT_BEST_SECOND_LEG_DATE,NEXT_BEST_SECOND_LEG_DEP_TIME,NEXT_BEST_SECOND_LEG_ARR_TIME
0,DL,ATL,"Atlanta, GA",AUS,"Austin, TX",12/1/2019,820,947,0.018167,0.008820,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,DL,AUS,"Austin, TX",SEA,"Seattle, WA",12/1/2019,614,858,12/2/2019,614,858
1,DL,ATL,"Atlanta, GA",AUS,"Austin, TX",12/1/2019,943,1113,0.008167,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,DL,AUS,"Austin, TX",SEA,"Seattle, WA",12/1/2019,614,858,12/2/2019,614,858
2,DL,ATL,"Atlanta, GA",AUS,"Austin, TX",12/1/2019,820,947,0.018167,0.008820,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,DL,AUS,"Austin, TX",SEA,"Seattle, WA",12/2/2019,614,858,12/3/2019,610,855
3,DL,ATL,"Atlanta, GA",AUS,"Austin, TX",12/1/2019,943,1113,0.008167,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,DL,AUS,"Austin, TX",SEA,"Seattle, WA",12/2/2019,614,858,12/3/2019,610,855
4,DL,ATL,"Atlanta, GA",BNA,"Nashville, TN",12/1/2019,930,936,0.000000,0.019373,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,DL,BNA,"Nashville, TN",SEA,"Seattle, WA",12/1/2019,700,1019,12/2/2019,700,1019
5,DL,ATL,"Atlanta, GA",BNA,"Nashville, TN",12/1/2019,815,822,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,DL,BNA,"Nashville, TN",SEA,"Seattle, WA",12/1/2019,700,1019,12/2/2019,700,1019
6,DL,ATL,"Atlanta, GA",BNA,"Nashville, TN",12/1/2019,930,936,0.000000,0.019373,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,DL,BNA,"Nashville, TN",SEA,"Seattle, WA",12/2/2019,700,1019,12/3/2019,715,1036
7,DL,ATL,"Atlanta, GA",BNA,"Nashville, TN",12/1/2019,815,822,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,DL,BNA,"Nashville, TN",SEA,"Seattle, WA",12/2/2019,700,1019,12/3/2019,715,1036
8,DL,ATL,"Atlanta, GA",BOS,"Boston, MA",12/1/2019,856,1126,0.023877,0.009181,0.007850,0.000000,0.000000,0.000000,0.000000,0.009761,DL,BOS,"Boston, MA",SEA,"Seattle, WA",12/1/2019,640,1015,12/1/2019,1720,2108
9,DL,ATL,"Atlanta, GA",BOS,"Boston, MA",12/1/2019,730,958,0.008991,0.000000,0.000000,0.000000,0.000000,0.009872,0.000000,0.000000,DL,BOS,"Boston, MA",SEA,"Seattle, WA",12/1/2019,640,1015,12/1/2019,1720,2108


In this example "arr_no_later" is set to the following day, so more results are presented.

In [ ]:
origin = 'SAN'
destination = 'ANC'
flight_date = '12/3/2019'
arr_no_later_date = '12/4/2019'

dne = '1'
dnl = '2359'
ane = '1'
anl = '2359'
tc = 45

df = itineraryBuilder('faa_2019_12', origin, destination, flight_date, arr_no_later_date, tc, dne, dnl, ane, anl); df.head(20)

You can also order by trip duration as shown. Other options include to sort by "earliest_arrival" and "min_connection_time".

In [ ]:
df = itineraryBuilder('faa_2019_12', origin, destination, flight_date, arr_no_later_date, tc, dne, dnl, ane, anl, orderby='duration'); df.head(10)

### Query Flights Function Call

Query flights includes many of the same parameters as Itinerary Builder and is in fact called by Itinerary Builder. This returns the initial query with the dates and times as string values.

In [ ]:
origin = 'OMA'
destination = 'ABQ'
flight_date = '12/1/2019'

dne = '1'
dnl = '2359'
ane = '1207'
anl = '534'
ane_date = '12/1/2019'
anl_date = '12/2/2019'

df = queryFlights('faa_2019_12', origin, destination, flight_date, ane_date, anl_date, dne, dnl, ane, anl); df

### getValidDestinations function call

Use this function to get a list of locations which can be reached with exactly two flights from the origin city, within a two day period.

In [ ]:
df2 = getValidDestinations('faa_2019_12', origin, flight_date); df2